# 리소스 제거
❗ 이 노트북은 데이터 변환 및 피쳐 프로젝트에서 생성된 모든 리소스를 제거합니다. 다음 코드 셀은 다음을 수행합니다.
- Studio 환경에서 프로비저닝한 프로젝트를 영구적으로 삭제
- 피쳐 그룹 또는 그룹을 영구적으로 삭제
- 프로젝트 프로비저닝된 S3 버킷을 영구적으로 삭제
- 프로젝트 관련 접두사 아래에 있는 S3 버킷의 객체를 영구적으로 삭제
- 관련 스택 제거

<div class="alert alert-info"> 💡 <strong> 소스를 삭제합니다. 삭제할 리소스의 이름을 다시 확인하십시오! </strong>
</div>

In [2]:
import sagemaker
import boto3
import time
import json
import os

In [3]:
# load environment variables from %store
%store -r 

In [4]:
%store

Stored variables and their in-db values:
abalone_dataset_file_name             -> 'abalone.csv'
abalone_dataset_local_url             -> '../dataset/abalone.csv'
data_bucket                           -> 'sagemaker-ap-southeast-2-189546603447'
domain_id                             -> 'd-bqnj3arguz1d'
dw_flow_file_url                      -> 's3://sagemaker-ap-southeast-2-189546603447/featur
dw_output_name                        -> 'a59ef526-30d3-481a-ac31-cb4f978f8413.default'
execution_role                        -> 'arn:aws:iam::189546603447:role/MLOps-custom-templ
feature_group_name                    -> 'FG-abalone-05-09-44-11-a856c9ef'
s3_data_prefix                        -> 'sagemaker-ap-southeast-2-189546603447/feature-sto
s3_flow_prefix                        -> 'sagemaker-ap-southeast-2-189546603447/feature-sto
s3_fs_query_output_prefix             -> 'sagemaker-ap-southeast-2-189546603447/feature-sto
s3_input_data_prefix                  -> 'sagemaker-ap-southeast-2-189546603

In [5]:
try:
    s3_data_prefix
    s3_flow_prefix
    s3_input_data_prefix
    s3_fs_query_output_prefix
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-setup.ipynb and 01-feature-store-ingest-pipeline notebooks")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

# 1. 프로젝트 삭제

In [7]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

These projects have been created by domain d-bqnj3arguz1d: [
  {
    "ProjectName": "s3-fs-ingest-05-11-54-15",
    "ProjectId": "p-8wp0z5luvu1v"
  }
]


In [8]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

Are you sure you want to delete this project: s3-fs-ingest-05-11-54-15? (y/n)


 y


***************************************
The following projects will be deleted:
[
  {
    "ProjectName": "s3-fs-ingest-05-11-54-15",
    "ProjectId": "p-8wp0z5luvu1v"
  }
]
***************************************


<div class="alert alert-info"> 💡 <strong> 프로젝트를 삭제합니다. 모든 프로젝트 CodeCommit 코드 리포지토리 및 CI/CD CodePipeline 파이프라인이 삭제됩니다! </strong>
</div>

In [9]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

Deleting project s3-fs-ingest-05-11-54-15:{'ResponseMetadata': {'RequestId': 'fd5f57ec-c751-4996-8ca4-0b315008e613', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fd5f57ec-c751-4996-8ca4-0b315008e613', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sun, 06 Feb 2022 11:32:41 GMT'}, 'RetryAttempts': 0}}


# 2. 피쳐 스토어의 피쳐 그룹 삭제

In [10]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [11]:
feature_groups

[{'FeatureGroupName': 'FG-abalone-05-09-44-11-a856c9ef',
  'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-2:189546603447:feature-group/fg-abalone-05-09-44-11-a856c9ef',
  'CreationTime': datetime.datetime(2022, 2, 5, 10, 19, 57, 259000, tzinfo=tzlocal()),
  'FeatureGroupStatus': 'Created'}]

In [12]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")


Are you sure you want to delete this feature group: FG-abalone-05-09-44-11-a856c9ef? (y/n)


 y


********************************************
The following feature groups will be deleted:
[
  "FG-abalone-05-09-44-11-a856c9ef"
]
********************************************


In [13]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> 다음 코드 셀은 선택한 기능 그룹을 삭제합니다!</strong>
</div>

In [15]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

Deleting the feature group: FG-abalone-05-09-44-11-a856c9ef
all feature store objects under s3://sagemaker-ap-southeast-2-189546603447/189546603447/sagemaker/ap-southeast-2/offline-store/FG-abalone-05-09-44-11-a856c9ef-1644056397/data will be deleted!
Are you sure you want to these objects ? (y/n)


 y


delete: s3://sagemaker-ap-southeast-2-189546603447/189546603447/sagemaker/ap-southeast-2/offline-store/FG-abalone-05-09-44-11-a856c9ef-1644056397/data/year=2022/month=02/day=05/hour=13/part-00000-05207ee9-92c1-4fbd-88ec-4a0ddaa831ac.c000.parquet
delete: s3://sagemaker-ap-southeast-2-189546603447/189546603447/sagemaker/ap-southeast-2/offline-store/FG-abalone-05-09-44-11-a856c9ef-1644056397/data/year=2022/month=02/day=05/hour=13/part-00001-05207ee9-92c1-4fbd-88ec-4a0ddaa831ac.c000.parquet
delete: s3://sagemaker-ap-southeast-2-189546603447/189546603447/sagemaker/ap-southeast-2/offline-store/FG-abalone-05-09-44-11-a856c9ef-1644056397/data/year=2022/month=02/day=05/hour=13/part-00006-05207ee9-92c1-4fbd-88ec-4a0ddaa831ac.c000.parquet
delete: s3://sagemaker-ap-southeast-2-189546603447/189546603447/sagemaker/ap-southeast-2/offline-store/FG-abalone-05-09-44-11-a856c9ef-1644056397/data/year=2022/month=02/day=05/hour=13/part-00015-05207ee9-92c1-4fbd-88ec-4a0ddaa831ac.c000.parquet
delete: s3://sag

# 3. 프로젝트 프로비저닝된 S3 버킷 삭제

<div class="alert alert-info"> 💡 <strong> 
다음 코드 셀은 프로젝트에서 생성된 모든 S3 버킷을 삭제합니다!</strong>
</div>

In [16]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-cp-{p['ProjectName']}-{p['ProjectId']}")
    print(f"sagemaker-ct-{p['ProjectName']}-{p['ProjectId']}")

*****************************************************
The following S3 buckets will be removed permanently!
*****************************************************
sagemaker-cp-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v
sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v


In [17]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-cp-{p['ProjectName']}-{p['ProjectId']} --force    
    !aws s3 rb s3://sagemaker-ct-{p['ProjectName']}-{p['ProjectId']} --force

delete: s3://sagemaker-cp-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/sagemaker-s3-fs-inge/DataLoadPi/CSTyYbz
remove_bucket: sagemaker-cp-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v
delete: s3://sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/AWSLogs/189546603447/CloudTrail/ap-southeast-2/2022/02/05/189546603447_CloudTrail_ap-southeast-2_20220205T1325Z_6Wa4MojVlsjBIB8h.json.gz
delete: s3://sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/AWSLogs/189546603447/CloudTrail/
delete: s3://sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/AWSLogs/189546603447/CloudTrail/ap-southeast-2/2022/02/05/189546603447_CloudTrail_ap-southeast-2_20220205T1200Z_qwx8kH1YLHn0Ed5C.json.gz
delete: s3://sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/AWSLogs/189546603447/CloudTrail/ap-southeast-2/2022/02/05/189546603447_CloudTrail_ap-southeast-2_20220205T1340Z_mSv1G91aVAYGuxFC.json.gz
delete: s3://sagemaker-ct-s3-fs-ingest-05-11-54-15-p-8wp0z5luvu1v/AWSLogs/189546603447/CloudTrail/ap-southeast-2/2022/

# 4. S3 데이터 버킷에서 프로젝트 관련 객체 제거

<div class="alert alert-info"> 💡 <strong> 
다음 코드 셀은 지정된 S3 접두사 아래의 모든 객체를 삭제합니다.!</strong>
</div>

In [18]:
prefixes_to_delete = [
    s3_data_prefix,
    s3_flow_prefix,
    s3_input_data_prefix,
    s3_fs_query_output_prefix
]

In [19]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(p)

************************************************************************
All objects under the following S3 prefixes will be removed permanently!
************************************************************************
sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dataset/
sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dw-flow/
sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/landing-zone/
sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/fs_query_results/


In [20]:
for p in prefixes_to_delete:
    !aws s3 rm s3://{p} --recursive

delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dataset/abalone.data
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dataset/abalone.csv
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dataset/abalone.names
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/dw-flow/dw2-flow-05-09-44-11-a856c9ef.flow
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/landing-zone/abalone-05-13-16-54.csv
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/fs_query_results/cf6643ba-32fc-4921-9d76-b5f79c5ebc49.csv
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/fs_query_results/1d4fe164-1fbb-4772-8459-e9ac721f4504.csv.metadata
delete: s3://sagemaker-ap-southeast-2-189546603447/feature-store-ingestion-pipeline/fs_query_results/1d4fe164-1fbb-4772-8459-e9ac721f4504.csv
de

## sm-project-sc-portfolio 스택 제거

In [2]:
SC_PORTFOLIO_STACK_NAME='sm-project-sc-portfolio'

In [5]:
%%sh -s "{SC_PORTFOLIO_STACK_NAME}" 

sc_portfolio_stack_name=$1
export SM_SC_FS_INGESTION_POLICY_ARN=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`FSIngestionProductPolicyArn`].OutputValue')

echo 'SM_SC_FS_INGESTION_POLICY_ARN' : $SM_SC_FS_INGESTION_POLICY_ARN
    
export SM_EXECUTION_ROLE_POLICY_ARN=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`AmazonSageMakerExecutionRolePolicyArn`].OutputValue')

echo ''
echo 'SM_EXECUTION_ROLE_POLICY_ARN' : $SM_EXECUTION_ROLE_POLICY_ARN    
    
export SM_EXECUTION_ROLE_NAME=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`AmazonSageMakerExecutionRoleName`].OutputValue')

echo ''
echo 'SM_EXECUTION_ROLE_NAME' : $SM_EXECUTION_ROLE_NAME    

echo "Policies are detached"    
aws iam detach-role-policy \
    --role-name AmazonSageMakerServiceCatalogProductsLaunchRole \
    --policy-arn $SM_SC_FS_INGESTION_POLICY_ARN

aws iam detach-role-policy \
    --role-name  $SM_EXECUTION_ROLE_NAME \
    --policy-arn $SM_EXECUTION_ROLE_POLICY_ARN

echo "Portfolio stack is deleted"    
aws cloudformation delete-stack --stack-name $sc_portfolio_stack_name



SM_SC_FS_INGESTION_POLICY_ARN :

SM_EXECUTION_ROLE_POLICY_ARN :

SM_EXECUTION_ROLE_NAME :
Policies are detached
Portfolio stack is deleted



An error occurred (ValidationError) when calling the DescribeStacks operation: Stack with id sm-project-sc-portfolio does not exist

An error occurred (ValidationError) when calling the DescribeStacks operation: Stack with id sm-project-sc-portfolio does not exist

An error occurred (ValidationError) when calling the DescribeStacks operation: Stack with id sm-project-sc-portfolio does not exist


# 5. 제공된 amazon-sagemaker-reusable-components-package-cfn 스택 제거

In [6]:
%%sh
aws cloudformation delete-stack --stack-name amazon-sagemaker-reusable-components-package-cfn

# 6. 노트북 리소스 제거

In [7]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

# 7. 노트북 및 실행 중인 노트북 인스턴스를 종료

<div class="alert alert-info"> 💡 <strong>아래에서 노트북을 셧다운 해주세요. 
그리고 RUNNING INSTANCE 가 필요 없으면 역시 꺼주세요. 

</strong>
</div>


<img src="img/delete-resource.png" width=400>

# 8. Next
계속 리소스 제거를 위해서 여기로 이동해주세요. [README-Action](../README-Action.md)